# 8.2 DeepDream

The DeepDream algorithm [[ref]](http://mng.bz/xXlM) is almost identical to the convnet filter visualization technique introduced in chapter 5, consisting of running a convnet in reverse: doing gradient ascent on the input to the convnet in order to maximize the activation of a specific filter in an upper layer of the convnet. 

The difference of DeepDream are:
- Here, you try to maximize the activation of entire layers rather than that of a specific filter, thus mixing together visualization of large numbers of features at once
- You start not from blank, slightly noisy input, but rather from an existing image, thus the resulting effects latch on to preexisting visual patterns, distorting elements of the image in a somewhat artistic fashion
- The input images are processed at different scales (called *octaves*), which improves the quality of the visualization

## 8.2.1 Implementing DeepDream in Keras

We'll start from a convnet pretrained on ImageNet. Since is it known that Inception produces nice-looking DeepDreams, we'll use Inception V3 that comes with Keras:

### L8.8 Loading the pretrained Inception V3 model

In [1]:
!pip install keras==2.0.8

     |████████████████████████████████| 276kB 2.7MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.8 which is incompatible.
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

In [4]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0) # to disable training operations

model = inception_v3.InceptionV3(weights='imagenet', include_top=False)






85319680/87910968 [============================>.] - ETA: 0sWARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.







Here we will try to simultaneously maximize the activation of all filters in a number of layers. Specifically, we'll maximize a weighted sum of the L2 norm of the activations of a set of high-level layers. 

High layers produce visual in which you can recognize some classes from ImageNet (for example birds and dogs). If you want to activate lower layers, it will result in geometric patters. 

Let's see an implementation for 4 layers:

### L8.9 Setting up the DeepDream configuration

In [5]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [0]:
# This is a dictionary mapping layer names to a coef. quantifying how much the 
# layer's activation contributes to the loss you'll seek to maximize.
# Note that the layer names are hardcoded in the build-in Inception V3 application.
# You can list all layer names using model.summary()
layer_contributions = {
    'mixed2': 1.0,
    'mixed3': 1.0,
    'mixed4': 0.1,
    'mixed5': 1.0,
    # 'mixed6': 0.2,
    # 'mixed7': 3.,
    # 'mixed8': 2.,
    # 'mixed9': 1.5,
}

Now we define a tensor that contains the loss: the weighted sum of the L2 norm of the activations of the layers in L8.9

### L8.10 Defining the loss the be maximized

In [66]:
# here we create a dict that maps layer names to layer instances
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# we define the loss by adding layer contributions to this scalar variable
loss = K.variable(0.)
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]
  # this retrieves the layer output
  activation = layer_dict[layer_name].output

  scaling = K.prod(K.cast(K.shape(activation), 'float32'))
  # this adds the L2 norm of the features of a layer to the loss
  # We avoid border artifacts by only involving nonborder pixels in the loss
  loss += coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

Next, we can set up the gradient-ascent process

### L8.11 Gradient-ascent process

In [0]:
# this tensor holds the generated image: the dream
dream = model.input

# this computes the gradients of the dream w.r.t. the loss
grads = K.gradients(loss, dream)[0]

# this normalizes the gradients (important trick)  
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# sets up a Keras function to retrieve the value of the loss and gradients, given an input image
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_values = outs[1]
  return loss_value, grad_values

# this function runs gradient ascent for a number of iterations
def gradient_ascent(x, iterations, step, max_loss=None):
  for i in range(iterations):
    loss_value, grad_values = eval_loss_and_grads(x)
    if max_loss is not None and loss_value > max_loss:
      break
    # print('...Loss value at {}: {}'.format(i, loss_value))
    x += step * grad_values
  return x

Now well go to the actual DeepDream algorithm. First we have to define a list of *scales* at which to process the images. Each successive scale is larger than the previous one by a factor of 1.4 (it's 40% larger).

For each step, from the smallest scale to the largets one, we run gradient ascent to maximize the loss we previously defined, at that scale. After each gradient ascent run, we upscale the resulting image by 40%.

To avoid losing a lot of image detail after each successive scale-up, we can use a simple trick: we'll re-inject the lost details back into the image, which is possible because we know what the original image should look like at the larger scale. 

- Given a small image size S and a larger image size L, we can compute the difference between the original image resized to size L and the original resized to size S, this difference quantifies the details lost when going from S to L.

### L8.12 Running gradient ascent over different successive scales

In [0]:
# Auxiliary functions

import scipy
from keras.preprocessing import image

import imageio

def resize_img(img, size):
  img = np.copy(img)
  factors = (1, 
             float(size[0]) / img.shape[1],
             float(size[1]) / img.shape[2],
             1)
  return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
  pil_img = deprocess_image(np.copy(img))
  # scipy.misc.imsave(fname, pil_img)
  imageio.imwrite(fname, pil_img)

def preprocess_image(image_path):
  img = image.load_img(image_path)
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = inception_v3.preprocess_input(img)
  return img

def deprocess_image(x):
  if K.image_data_format() == 'channels_first':
    x = x.reshape((3, x.shape[2], x.shape[3]))
    x = x.transpose((1, 2, 0))
  else:
    x = x.reshape((x.shape[1], x.shape[2], 3))
  x /= 2.
  x += 0.5
  x *= 255.
  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [69]:
cd /content/drive/My Drive/kaggle/

/content/drive/My Drive/kaggle


In [70]:
import numpy as np

step = 0.01
num_octave = 10
octave_scale = 1.3
iterations = 20

# If the loss grows larger than 10, we'll interrupt the gradient-ascent process to avoid artifacts
max_loss = 8.

base_image_path = 'm1.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
  shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
  successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
  print('Processing image shape', shape)
  img = resize_img(img, shape)
  img = gradient_ascent(img, iterations=iterations, step=step, max_loss=max_loss)
  
  upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
  
  same_size_original = resize_img(original_img, shape)
  
  lost_detail = same_size_original - upscaled_shrunk_original_img
  img += lost_detail
  
  shrunk_original_img = resize_img(original_img, shape)
  
  # save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
save_img(img, fname='final_dream.png')

Processing image shape (325, 434)
Processing image shape (423, 564)
Processing image shape (550, 734)
Processing image shape (716, 954)
Processing image shape (930, 1241)
Processing image shape (1210, 1613)
Processing image shape (1573, 2097)
Processing image shape (2044, 2726)
Processing image shape (2658, 3544)
Processing image shape (3456, 4608)
